# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
05/07/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from fonctions_BS import *

In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x000001FAE8F26300>
Using matplotlib backend: TkAgg


In [3]:
# PARAMETRE ACQUISITION
# S_force_bs =       # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure
S_camera = 98  # pix/mm

dossier = '2024.05.30'
imag = '/image_'
seuil = 120

In [4]:
# EXEMPLES

# I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE) # lecture d'une image en niveau de gris
# _, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY) # lecture d'image en noir et blanc selon le seuil défini
# I = I.astype(np.float32)

# xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche)
# I_rogn = rogner(I_rot, xy_anche)
# coord = bord_anche(I_rogn)
# Xv, Yv = deplacement_profil(I_rogn, seuil, coord)

# plt.imshow(I_rogn, cmap='gray')
# print(Xv, Yv)



# Traitement image

In [5]:
plt.close('all')


# def....(dossier, imag, seuil, nbre_y, nbre_z, pas_z, plot_figure=False)
#   return y, coefficients


# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE) # commencer par l'image de la poutre rigide définissant l'horizontalité
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche) # pas besoin de faire une rotation 
I_rogn = rogner(I, xy_anche)
coord_rogn = bord_anche(I_rogn)
print(coord_rogn)

# Nombre des points de mesures 
Ys = 7 # sur la largeur de l'anche, selon y
Zs = 20 # en appui selon z
pas_z = 0.05 # pas d'appui du catpeur selon z, en mm
# (w, h) = I_rogn.shape


plot_fig = True
# if plot_fig :
#         fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        
# Import data
for y in range(Ys):
    if plot_fig :
            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    with open(dossier + f'/profil{y}.txt', 'w') as file:
        for z in range(Zs):
            # chargement de l'image
            img_path = dossier + imag + f'{y}_{z}.jpg'
            if not os.path.exists(img_path):
                continue    # si l'image n'existe pas, le programme continue
            
            # traitement de l'image (mise en noir et blanc + rogner l'image suivant les bornes définies pour l'image d'initialisation)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)
            img = img.astype(np.float32)
            img_rogn =  rogner(img, xy_anche)
            
            # detection du profil
            Xv, Yv = detection_profil(img_rogn, seuil, coord_rogn)
            Yv_array = np.array(Yv)
            
            # masque pour suprimer les valeurs abérentes
            mask = ~np.isnan(Xv) & ~np.isnan(Yv_array) & ~np.isinf(Xv) & ~np.isinf(Yv_array)
            Xv_clean = Xv[mask]
            Yv_clean = Yv_array[mask]
            
            # grandeur physique
            Xv_physique = Xv_clean/S_camera
            Yv_physique = Yv_clean/S_camera
        
            # polyfit
            coefficients_fit = np.polyfit(Xv_physique, Yv_physique, 3)
            
            # FIGURE
            if plot_fig :
                axs[0].imshow(img_rogn, cmap='gray')
                axs[0].plot(Xv, Yv, '-', lw=2)
                axs[0].set_xlabel('y (mm)')
                axs[0].set_xlabel('z (mm)')
                axs[1].plot(Xv_physique, -np.polyval(coefficients_fit, Xv_physique), lw=2)
                axs[1].plot(Xv_physique, -Yv_physique, lw=2)
                axs[1].grid()
                
            #enregistrement dans un fichier des coefficients du modèle paramétrique de la parabole
            file.write(f'{y} {coefficients_fit[0]} {coefficients_fit[1]} {coefficients_fit[2]} \n')
    

In [35]:
plt.close('all')
seuil = 120
S_camera = 98  # pix/mm
# Initialisation sur une image

data_pos = pd.read_table(dossier + '/data_anche1.txt', sep=' ', header=1).values # contient : [y, z, offset, moyenne_brute, moyenne(N) en grandeur physique]
position_y = data_pos[:,0]
position_z = data_pos[:,1]
force=data_pos[:,4]



# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE) # commencer par l'image de la poutre rigide définissant l'horizontalité
# I = cv2.imread('a2_test.bmp', cv2.IMREAD_GRAYSCALE)
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche) # pas besoin de faire une rotation 
I_rogn = rogner(I, xy_anche)
coord_rogn = bord_anche(I_rogn)
# print(coord_rogn)

Ys = len(position_y)
Zs = len(position_z)
# Ys = 5
# Zs = 20
# Zs_sequence = np.arange(1, Zs+1)
pas_z = 0.05 #mm
nbr_slice = 20
(w, h) = I_rogn.shape


plot_fig = True
# if plot_fig :
#         fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# 2 Matrice (pente et ordonnée origine) de la taille du 'nombre de points d'appui du capteurs sur la largeur de l'anche' x 'nombre de slices/tranches/points d'obeservation'
A = np.zeros((Ys,Zs)) 
B = np.zeros((Ys,Zs))
# Import data
for y in range(Ys):
    if plot_fig :
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    data = np.zeros((Zs, (nbr_slice-1)))
    for z in range(Zs):
        img_path = dossier + imag + f'{y}_{z}.jpg' # recherhcer toutes les images
        if not os.path.exists(img_path):        # si le chemin n'est pas trouvé, cette image est ignorée et on passe à la suivante
            continue
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)    # lecture d'image en niveau de gris
        _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)  # mise en noir et blanc selon le seuil défini
        img = img.astype(np.float32)
        # _, img_rot = rotate(img, xy_anche)
        img_rogn =  rogner(img, xy_anche)       # rogner l'image proche de l'anche
        # print(img_rogn)
        
        Xv, Yv = detection_profil(img_rogn, seuil, coord_rogn) # détection de la pointe de l'anche image par image
        Yv_array = np.array(Yv)
        
        # se débarasser des valeurs abérentes (nan dans les Xv et Yv)
        mask = ~np.isnan(Xv) & ~np.isnan(Yv_array) & ~np.isinf(Xv) & ~np.isinf(Yv_array)
        Xv_clean = Xv[mask]
        Yv_clean = Yv_array[mask]
        # grandeur physique : on passe se pixel en mm
        Xv_physique = Xv_clean/S_camera
        Yv_physique = Yv_clean/S_camera
            
        # Etude de notre pointe par anche
        Xv_slice_center = []
        Yv_mean_per_slice = []
        slice_size = (np.abs(Xv_clean[-1]-Xv_clean[0]))/nbr_slice   # taille en pixel d'une tranche
        anche_center = (np.abs(Xv_clean[-1]-Xv_clean[0]))/2
        for n in range(nbr_slice-1):
            # début et fin de chaque slice par itération            
            slice_left = int(Xv_clean[0]+n*slice_size)
            slice_right = int(Xv_clean[0]+(n+1)*slice_size)
            Xv_slice_center.append(-anche_center + slice_left + np.abs(slice_left-slice_right)/2) # coordonnée centrale de la tranche
            Yv_slice = Yv_clean[slice_left:slice_right]
            Yv_mean_per_slice.append(np.mean(Yv_slice)) # déplacement moyen par slice (moyenne sur les Yv)
        
        # grandeur physique, mm
        Xv_slice_center_phys = np.array(Xv_slice_center)/S_camera
        Yv_mean_per_slice_phy = np.array(Yv_mean_per_slice)/S_camera   
        # print(Yv_mean_per_slice_phy.shape)  
        # print(Yv_mean_per_slice_phy)  
        data[z,:]=Yv_mean_per_slice_phy
        
        # traitement pour faire plus que le polyfit 
        if plot_fig :
            axs[0].imshow(img_rogn, cmap='gray')
            axs[0].plot(Xv, Yv, '-', lw=2)
            axs[0].set_xlabel('y (pixel)')
            axs[0].set_ylabel('z (pixel)')
            axs[1].plot(Xv_slice_center_phys, Yv_mean_per_slice_phy, '.-', lw=2)
            axs[1].set_xlabel('y (mm)')
            axs[1].set_ylabel('z (mm)')
            axs[1].grid('True')            
    print(len(data[deb:,slice]))
    print((force[deb:].shape))

    # plt.figure()
    for slice in range(nbr_slice):
        deb = 6
        droite = np.polyfit(data[deb:,slice], force,  1)
        a = droite[0]
        b = droite[1]
        A[y,slice]=a
        B[y,slice]=b
        if plot_fig :
            axs[2].plot(data[deb:,slice], force[deb:], '.')
            # axs[2].plot(np.unique(position_z)[deb:], np.polyval(droite, np.unique(position_z)[deb:]), linestyle='-', lw='1.5') 
            axs[2].set_ylabel('y (mm)')
            # axs[2].set_xlabel('z (mm)')  
            axs[2].grid('True') 
# print(f'{y} \npente = {A}\nordonnée = {B}')

    

266
(266,)


TypeError: expected x and y to have same length

In [68]:
A_flat = A.flatten()
B_flat = B.flatten()
AB_flat = np.concatenate((A_flat, B_flat))
AB_flat = ' '.join(map(str, AB_flat))
with open(dossier + '/data_MS_image.txt', 'w') as file:
    file.write(f'{AB_flat} \n')  # Écrit les matrices aplaties sur une seule ligne
    
test=np.genfromtxt(dossier + '/data_MS_image.txt', delimiter='')
print(test)



[0.00578643 0.0067631  0.00776567 0.00886297 0.01016911 0.01168012
 0.01357529 0.01581707 0.01839471 0.02173843 0.02544629 0.02956542
 0.03425693 0.03967143 0.04550684 0.0517487  0.05876243 0.06527761
 0.07024445 0.01044203 0.01203371 0.01367222 0.01537789 0.01733306
 0.01966547 0.0223262  0.02548217 0.02916744 0.03387718 0.03885064
 0.04412747 0.04971369 0.05601469 0.06213576 0.06768904 0.07259999
 0.07531749 0.0753435  1.84006523 1.87742992 1.91554692 1.94157434
 1.96165706 1.97823914 1.98170373 1.97836697 1.96073149 1.92500785
 1.87862039 1.88320859 1.89653285 1.86595269 1.86445989 1.85627962
 1.82224415 1.77402473 1.74011101 1.81299068 1.84744524 1.88204811
 1.9052751  1.92235091 1.93494132 1.93588439 1.92930216 1.90640753
 1.86480339 1.81325933 1.81576549 1.8289624  1.79672458 1.79765904
 1.79530753 1.77252411 1.74268913 1.73232825]
